In [1]:
import os
import pandas as pd
import numpy as np
import ast
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

In [2]:
df_hospitals = pd.read_csv('hospitals_kolkata.csv')
df_doctors = pd.read_csv('doctors_kolkata.csv')

In [3]:

print(f"Loaded {len(df_hospitals)} hospitals and {len(df_doctors)} doctors.")

Loaded 15 hospitals and 365 doctors.


In [4]:
def hospital_ann_score(features):
    """
    Simulates a trained Neural Network forward pass.
    Inputs: [Normalized_Distance, ICU_Free_Norm, Staff_Norm, Trauma_Center_Binary]
    """
    # Weights adjusted for your new features:
    # High priority on Trauma Center capability (0.9) and Distance (-0.6)
    weights = np.array([-0.6, 0.3, 0.1, 0.9]) 
    bias = 0.1
    score = np.dot(features, weights) + bias
    return max(0, score)

In [5]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

def setup_doctor_knowledge_base(df_doctors):
    print("--- Building Doctor Knowledge Base ---")
    docs = []

    for _, row in df_doctors.iterrows():
        content = (
            f"Doctor: {row['name']} (ID: {row['doctor_id']}). "
            f"Level: {row['seniority_level']}. "
            f"Specialty: {row['specialty']} ({row['sub_specialty']}). "
            f"Experience: {row['years_experience']} years. "
            f"Languages: {row['languages_spoken']}. "
            f"On Call: {row['on_call']}. "
            f"Prefers Trauma: {row['prefers_trauma_cases']}."
        )

        meta = {
            "hospital_id": row["hospital_id"],
            "doctor_id": row["doctor_id"],
        }

        docs.append(Document(page_content=content, metadata=meta))

    # 🔹 choose your model here:
    # "BAAI/bge-small-en"  = fast, English
    # "BAAI/bge-m3"        = multilingual, very good
    model_name = "BAAI/bge-small-en"   # or "BAAI/bge-m3"

    # BGE likes cosine similarity + normalized vectors
    encode_kwargs = {"normalize_embeddings": True}

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},   # change to "cuda" if you have GPU
        encode_kwargs=encode_kwargs,
    )

    vector_store = FAISS.from_documents(docs, embeddings)
    return vector_store

# usage:
vector_db = setup_doctor_knowledge_base(df_doctors)


--- Building Doctor Knowledge Base ---


--- Building Doctor Knowledge Base ---


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16252\2442373929.py:35: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


In [6]:
vector_db.save_local("doctor_faiss_index")

In [7]:
class AgentState(TypedDict):
    accident_location: tuple 
    accident_type: str 
    selected_hospital_id: str
    selected_hospital_name: str
    final_plan: str

In [8]:
def hospital_agent(state: AgentState):
    """
    Finds best hospital using 'icu_beds_free', 'facility_trauma_center', etc.
    """
    print(f"\n--- [Hospital Agent] Scanning for: {state['accident_type']} ---")
    
    user_lat, user_lng = state['accident_location']
    scored_hospitals = []

    for _, row in df_hospitals.iterrows():
        # 1. Distance
        dist = np.sqrt((row['lat'] - user_lat)**2 + (row['lng'] - user_lng)**2)
        norm_dist = min(dist * 10, 1.0)
        
        # 2. Features (Mapped to your new headers)
        # ICU Availability
        icu_free = row['icu_beds_free'] if pd.notna(row['icu_beds_free']) else 0
        norm_icu = min(icu_free / 20, 1.0) # Normalize (assuming 20 is a good number)
        
        # Staffing
        staff_count = row['staff_doctors_planned'] if pd.notna(row['staff_doctors_planned']) else 0
        norm_staff = min(staff_count / 50, 1.0)
        
        # Trauma Center Capability
        # Check if accident is trauma-related AND facility has trauma center
        is_trauma_accident = 'CRASH' in state['accident_type'].upper() or 'TRAUMA' in state['accident_type'].upper()
        
        # Using the boolean column directly
        has_trauma_center = 1.0 if row['facility_trauma_center'] == True else 0.0
        
        # If it's a trauma accident, having a trauma center is crucial (1.0). If not, it's neutral (0.0).
        trauma_score = has_trauma_center if is_trauma_accident else 0.5
        
        # 3. ANN Scoring
        features = np.array([norm_dist, norm_icu, norm_staff, trauma_score])
        score = hospital_ann_score(features)
        
        scored_hospitals.append({
            "id": row['hospital_id'],
            "name": row['hospital_name'], 
            "score": score,
            "dist": dist
        })
    
    # Sort and Select
    best_hospital = sorted(scored_hospitals, key=lambda x: x['score'], reverse=True)[0]
    
    print(f"--- [Hospital Agent] Selected: {best_hospital['name']} (Score: {best_hospital['score']:.2f}) ---")
    return {
        "selected_hospital_id": best_hospital['id'],
        "selected_hospital_name": best_hospital['name']
    }

In [ ]:
import os
import json
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import for modern LangChain
from dotenv import load_dotenv  # <--- NEW IMPORT

# Load variables from .env file into the environment
load_dotenv()

# --- 1. LOAD THE SAVED VECTOR DATABASE ---
# We must use the exact same model config used during the save step
print("--- Loading Vector Database from Disk ---")

embedding_model_name = "BAAI/bge-small-en"
encode_kwargs = {"normalize_embeddings": True}

# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cpu"}, 
    encode_kwargs=encode_kwargs
)

# Load the FAISS index from your folder
try:
    vector_db = FAISS.load_local(
        "doctor_faiss_index",  # Folder name containing index.faiss and index.pkl
        embeddings, 
        allow_dangerous_deserialization=True # Required for loading pickle files you created
    )
    print("✅ Vector DB loaded successfully.")
except Exception as e:
    print(f"❌ Error loading Vector DB: {e}")
    print("Ensure the folder 'doctor_faiss_index' exists in this directory.")
    # Stop execution if DB fails to load, as the agent depends on it
    raise e 


# --- 2. DOCTOR AGENT FUNCTION ---

def doctor_agent(state: AgentState):
    """
    Finds doctor using the loaded RAG Vector DB and ensures Hospital Name is in the output.
    """
    hosp_id = state['selected_hospital_id']
    hosp_name = state['selected_hospital_name'] 
    
    print(f"--- [Doctor Agent] Querying staff at {hosp_id} ({hosp_name}) ---")
    
    # 1. RETRIEVAL
    # Use the loaded vector_db to create a retriever
    retriever = vector_db.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "filter": {"hospital_id": hosp_id}, # Strict filter for the selected hospital
        },
    )
    
    query = f"""
    Accident: {state['accident_type']}
    Find a doctor with high seniority, trauma preference, and On Call=True.
    """
    
    # Fetch documents
    # invoke() returns a list of Documents in standard LangChain retrievers
    docs = retriever.invoke(query)
    
    # Debug: Verify we actually found doctors
    if not docs:
        print(f"⚠️ Warning: No doctors found for hospital {hosp_id} in Vector DB.")

    # 2. GENERATION (GROQ)
    llm = ChatGroq(
        temperature=0,
        model="llama-3.1-8b-instant",
        api_key=os.getenv("GROQ_API") # Your Key
    )
    
    context_text = "\n\n".join([d.page_content for d in docs])
    
    prompt = f"""
    Emergency Type: {state['accident_type']}
    Selected Hospital: {hosp_name}

    Candidate Doctors (Retrieved from Database):
    {context_text}

    Task:
    - Pick the BEST doctor based on availability and specialty.
    - Return a VALID JSON object (no markdown formatting, just raw JSON) with this structure: 
      {{
        "recommended_hospital": "{hosp_name}", 
        "target_doctor": "Name",
        "doctor_specialty": "Specialty",
        "status": "ON_CALL",
        "action": "Prepare OR",
        "reason": "One short sentence explanation"
      }}
    """
    
    response = llm.invoke(prompt)
    return {"final_plan": response.content}

--- Loading Vector Database from Disk ---
✅ Vector DB loaded successfully.


In [10]:
workflow = StateGraph(AgentState)
workflow.add_node("find_hospital", hospital_agent)
workflow.add_node("find_doctor", doctor_agent)
workflow.set_entry_point("find_hospital")
workflow.add_edge("find_hospital", "find_doctor")
workflow.add_edge("find_doctor", END)
app = workflow.compile()

In [11]:
inputs = {
    "accident_location": (22.645284, 88.378688), 
    "accident_type": "Cardiac arrest on roadway — Unconscious, not breathing"
}

result = app.invoke(inputs)

print("\n================ ORCHESTRATION RESULT ================\n")

print(f"🏥 RECOMMENDED HOSPITAL: {result['selected_hospital_name']}")
print(f"📍 Hospital ID: {result['selected_hospital_id']}")
print("-" * 40)


print("👨‍⚕️ DOCTOR & ACTION PLAN:")
print(result['final_plan'])


--- [Hospital Agent] Scanning for: Cardiac arrest on roadway — Unconscious, not breathing ---
--- [Hospital Agent] Selected: CityCare Hospital WA (Score: 0.89) ---
--- [Doctor Agent] Querying staff at HOSP_004 (CityCare Hospital WA) ---

================ ORCHESTRATION RESULT ================

🏥 RECOMMENDED HOSPITAL: CityCare Hospital WA
📍 Hospital ID: HOSP_004
----------------------------------------
👨‍⚕️ DOCTOR & ACTION PLAN:
```json
{
  "recommended_hospital": "CityCare Hospital WA",
  "target_doctor": "Aditya Singh",
  "doctor_specialty": "CARDIAC (INTERVENTIONAL_CARDIOLOGY)",
  "status": "ON_CALL",
  "action": "Prepare OR",
  "reason": "Aditya Singh is a senior consultant with 15 years of experience in interventional cardiology and is available on call."
}
```

This recommendation is based on the following criteria:

- **Availability**: Both Aditya Singh and Nikita Singh are available on call.
- **Specialty**: Aditya Singh has a specialty in interventional cardiology, which is mor